# Import Libraries

In [252]:
#pip install bs4
#import bs4
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import pandas as pd

from datetime import datetime,timedelta

import time
 


In [253]:
from http.client import IncompleteRead

# Read csv file

In [254]:
df = pd.read_excel("D:\Research Project\Datasets\Flood intensity\FloodArchive.xlsx")
df.head()

,ID,GlideNumber,Country,OtherCountry,long,lat,Area,Began,Ended,Validation,Dead,Displaced,MainCause,Severity
0,1,0,Algeria,0,5.23026,35.81420,92615.67,1985-01-01,1985-01-05,News,26,3000,Heavy rain,1.0
1,2,0,Brazil,0,-45.34890,-18.71110,678498.82,1985-01-15,1985-02-02,News,229,80000,Heavy rain,2.0
2,3,0,Phillipines,0,122.97400,10.02070,12846.03,1985-01-20,1985-01-21,News,43,444,Torrential rain,1.0
3,4,0,Indonesia,0,124.60600,1.01489,16542.12,1985-02-04,1985-02-18,News,21,300,Torrential rain,1.0
4,5,0,Mozambique,0,32.34910,-25.86930,20082.21,1985-02-09,1985-02-11,News,19,0,Heavy rain,2.0


# Function to extract weather URL

In [255]:
def getdetails(URL,began_date):
    dt_object = began_date.date()
    dt_object1 = dt_object-timedelta(days=2) # this is for day-2's values
    dt_object = dt_object1
    

    req = Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    value1 = str(soup.find_all("div", {"class":"station-name ng-star-inserted"}))
    result_list = re.sub(r"^.+?(?=history)", "", value1)
    
    split_string = result_list.split("date", 1)
    substring = split_string[0]
    
    hist_url = 'https://www.wunderground.com/'+substring+'date/'+str(dt_object)+'/'
    return hist_url

# Web scraping weather URL based on coordinates and Date

In [256]:

url= "https://www.wunderground.com/weather/"
for i in range(len(df)):
    # adding this line as internet got interrupted inbetween
    if i > 4163:
        new_url = url + str(df.loc[i, "lat"]) + ',' + str(df.loc[i, "long"]) +'/'
        df.loc[i,"coordinate url"]=new_url
        try :
            val= getdetails(new_url,df.loc[i, "Began"])
            df.loc[i,"began date url"]=val


        except IncompleteRead:
            # reconnect and keep trucking
            continue 

        if i % 50 ==0:
                print(f"{i} rows iterated")
                # Wait for 5 seconds
                #time.sleep(5)




4200 rows iterated
4250 rows iterated
4300 rows iterated
4350 rows iterated
4400 rows iterated
4450 rows iterated
4500 rows iterated
4550 rows iterated
4600 rows iterated
4650 rows iterated
4700 rows iterated
4750 rows iterated
4800 rows iterated
4850 rows iterated
4900 rows iterated


# Export Dataframe to extract weather details via UiPath

In [259]:
#df
df.to_csv('D:\Research Project\Datasets\Flood intensity\data url\csv files\date2_url_from_4163.csv')
